# CNN/GNN/LSTM Model Test

## Import dependencies and data

In [ ]:
# import dependencies
import os
import pandas as pd


import matplotlib.pyplot as plt
#import tensorflow as tf
#import keras

In [70]:
# define relevant paths
home_path = os.path.dirname(os.getcwd())
data_path = home_path + '\\data\\'
plot_path = home_path + '\\plotting\\plots\\'

In [71]:
# get merged data
data = pd.read_csv(
    data_path + 'data_merged.csv',
    header=[0, 1],
    index_col=[0, 1, 2, 3]
)

# convert index.date col to datetime
#data.index = pd.to_datetime(data.index.values)
data.index = data.index.set_levels(
    levels=pd.to_datetime(data.index.get_level_values(3).values),
    level=3
)

In [72]:
# define X vars
x_vars = [
    ('pressure', 'DC_GP101'),
    ('pressure', 'SMHI'),
    ('flow', 'TA01_GP101_default'),
    ('flow', 'FF01_GP101_default'),
    ('temperatures', 'DC_GT401_GM401'),
    ('temperatures', 'TA01_GT10X_GM10X'),
    ('temperatures', 'DC_GT301_damped'),
    ('temperatures', 'DC_GT301_outdoor'),
    ('humidity', 'TA01_GT10X_GM10X'),
    ('humidity', 'DC_GT401_GM401'),
    ('humidity', 'SMHI'),
    ('setpoints', 'DC_GP101_default'),
    ('setpoints', 'TA01_GT10X_GM10X_default')
]

# filter columns to keep only x_vars
data = data[x_vars].copy()

In [74]:
# print number of NAs
print('Number of NAs\n')
print(data.isna().sum())
print('\n\n')

# check for NaNs
nadat = data.droplevel(level=0, axis=1)[[
    'TA01_GP101_default', 
    'FF01_GP101_default',
    'DC_GP101_default',
    'TA01_GT10X_GM10X_default'
]]

## display NaNs across days
#print(nadat.isna().groupby(['month', 'day']).sum())
##print(nadat.isna().groupby('day').sum())

## remove NANs
#data = data.dropna(
#    how='any', 
#    subset=x_vars
#)

Number of NAs

category      sensor_ID               
pressure      DC_GP101                    0
              SMHI                        0
flow          TA01_GP101_default          0
              FF01_GP101_default          0
temperatures  DC_GT401_GM401              0
              TA01_GT10X_GM10X            0
              DC_GT301_damped             0
              DC_GT301_outdoor            0
humidity      TA01_GT10X_GM10X            0
              DC_GT401_GM401              0
              SMHI                        0
setpoints     DC_GP101_default            0
              TA01_GT10X_GM10X_default    0
dtype: int64





## Process / transform variables 

In [ ]:
# outdoors pressure in hPA...

# TA02 not needed

# Use temperature DIfF w.r.t. setpoint

# Use rolling avg. f. humidity


## Split data

In [ ]:
def seq_extract(x, n_steps):
    """
    Create sequences f. CNN, incl. channel dim.
    """
    n = len(x)
    
    sequences = np.stack([
        x[i:i+n_steps] for i in range(n - n_steps)
    ])
    
    return sequences[..., np.newaxis]

def data_split(data, var):
    """
    Split and whiten data. Using training mean and s.d. also for test data.
    """
    x_train = data[data.index.get_level_values(0) != 2 ][var].values
    x_test = data[data.index.get_level_values(0) == 2][var].values
    
    # Whiten both sets w. training params
    mean = np.mean(x_train)
    std = np.std(x_train)
    
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std

    # get sequences
    x_train = seq_extract(
        x=x_train,
        n_steps=120
    )

    x_test = seq_extract(
        x=x_test,
        n_steps=120
    )
    
    return x_train, x_test
    

## Define model